<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/langchain_GPT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env --quiet

In [ ]:
!pip install openai -q

In [ ]:
import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%cd /content/
!wget "https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1" -O pg_essay.txt
#!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

In [ ]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

In [5]:
modelid=modellist.data[11].id
modelid

'gpt-4o'

In [6]:
def chat_gpt_textgen(prompt):
    response = client.chat.completions.create(
      #model="gpt-3.5-turbo",
      #model="gpt-4",
      #model="gpt-4-turbo-preview",
      model=modelid,
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
      ]
    )

    print(response.choices[0].message.content)

In [7]:
prompt = "Who won the world series in 2020?"
chat_gpt_textgen(prompt)

The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays in six games to secure the championship. This victory marked the Dodgers' first World Series title since 1988.


In [ ]:
!pip install langchain -q
!pip install langchain-core -q
!pip install langchain-community -q
!pip install langchain-anthropic -q

In [9]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/content/pg_essay.txt')
data = loader.load()

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = splitter.split_documents(data)

In [ ]:
!pip install tiktoken -q

In [ ]:
!pip install faiss-gpu -q
!pip install faiss-cpu -q

In [ ]:
#from langchain_community.embeddings import CohereEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

retriever = vector.as_retriever()

# create a chain to answer questions
llm=OpenAI(model_name='gpt-3.5-turbo-instruct',max_tokens=1512,temperature=0.9)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)


In [ ]:
#prompt="How AWS has evolved?"
prompt="What did the author do growing up?"
result = qa({"query": prompt})

In [22]:
print()
#print('chain to answer questions')
print("-" * 80)
print()
result = qa({"query": prompt})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')
print("-" * 80)


--------------------------------------------------------------------------------

Query: What did the author do growing up?

Result: 
The author wrote short stories and worked on programming, specifically on the IBM 1401 which was located in the basement of their school. They also started taking art classes at Harvard and eventually dropped out of a PhD program in computer science to focus on painting. They also worked on a programming project called Bel.

Context Documents: 
page_content='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data process